# DAG: Thesis-Recommenders-Upgrade

Analisis de resultados de cada tarea que constrituye el dag _Thesis-Recommenders-Upgrade_.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
BASE_PATH = '../'
LIB_PATH  = f'{BASE_PATH}/rec-sys-client-lib/src'
DATA_PATH ='/var/tmp/rec-sys-client'

In [3]:
import sys
sys.path.append(LIB_PATH)

from pytorch_common.util import LoggerBuilder

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from bunch import Bunch

from recsys import DomainContext

In [4]:
LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [5]:
ctx = DomainContext(cfg_path = f'{BASE_PATH}/rec-sys-client-lib/config.yml')

2023-08-15 19:52:09,325 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


{'temp_path': '/var/tmp/rec-sys-client', 'weights_path': '/home/adrian/development/personal/maestria/thesis-paper/rec-sys-client-lib/weights', 'api': {'host': 'http://recsys.ddns.net:8000', 'token': 'e3ff025094fe0ee474501bbeda0a2a44e80230c1'}}


2023-08-15 19:52:09,512 - INFO - Use pytorch device: cuda
2023-08-15 19:52:09,512 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L12-v2
2023-08-15 19:52:09,712 - INFO - Use pytorch device: cuda
2023-08-15 19:52:09,712 - INFO - Load pretrained SentenceTransformer: multi-qa-mpnet-base-dot-v1
2023-08-15 19:52:10,295 - INFO - Use pytorch device: cuda
2023-08-15 19:52:10,296 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-08-15 19:52:10,863 - INFO - Use pytorch device: cuda


In [8]:
ctx.similarity_matrix_repository.find()

2023-08-15 19:52:31,646 - INFO - HTTP Request: GET http://recsys.ddns.net:8000/api/similarity-matrix/?offset=0&limit=100000&name=&type=&description=&version= "HTTP/1.1 200 OK"
2023-08-15 19:52:31,650 - INFO - Page 2/1 - Similarity_matrix 12/12
2023-08-15 19:52:31,650 - INFO - 12 Total Similarity_matrix 


[{
     "id": 51,
     "name": "NMF-user-to-user",
     "type": "SimilarityMatrixType.USER_TO_USER",
     "description": "NMF-user-to-user",
     "version": 69
 },
 {
     "id": 52,
     "name": "SVD-user-to-user",
     "type": "SimilarityMatrixType.USER_TO_USER",
     "description": "SVD-user-to-user",
     "version": 56
 },
 {
     "id": 53,
     "name": "NMF-item-to-item",
     "type": "SimilarityMatrixType.ITEM_TO_ITEM",
     "description": "NMF-item-to-item",
     "version": 68
 },
 {
     "id": 54,
     "name": "SVD-item-to-item",
     "type": "SimilarityMatrixType.ITEM_TO_ITEM",
     "description": "SVD-item-to-item",
     "version": 56
 },
 {
     "id": 55,
     "name": "all-MiniLM-L6-v2-item-to-item",
     "type": "SimilarityMatrixType.ITEM_TO_ITEM",
     "description": "all-MiniLM-L6-v2-item-to-item",
     "version": 0
 },
 {
     "id": 56,
     "name": "all-MiniLM-L12-v2-item-to-item",
     "type": "SimilarityMatrixType.ITEM_TO_ITEM",
     "description": "all-MiniLM-L12-v2-i

In [6]:
ctx.similarity_cell_repository.describe

['add_many', 'find', 'remove']

In [7]:
help(ctx.similarity_cell_repository.find)

Help on method find in module recsys.repository.entity_repository:

find(query={}, page_size=100000) method of recsys.repository.similarity_cell_repository.SimilarityCellRepository instance
    Allows to query entities by ani entity field.
    
    Args:
        query (dict, optional): A dict of field_name: value pairs. Defaults to {}.
        page_size (int, optional): Page size. Defaults to 100000.
    
    Returns:
        a dict list: A lost of dict. Each dist represent an entity or model like an interaction, a similar matrix, etc...



In [37]:
rows = ctx.similarity_cell_repository.find(query={'matrix':'61'})

2023-08-15 20:03:16,339 - INFO - HTTP Request: GET http://recsys.ddns.net:8000/api/similarity-matrix-cells/?offset=0&limit=100000&row=&column=&matrix=61 "HTTP/1.1 200 OK"
2023-08-15 20:03:17,478 - INFO - Page 1/2 - Similarity_cells 100000/107415
2023-08-15 20:03:18,051 - INFO - HTTP Request: GET http://recsys.ddns.net:8000/api/similarity-matrix-cells/?offset=100000&limit=100000&row=&column=&matrix=61 "HTTP/1.1 200 OK"
2023-08-15 20:03:18,144 - INFO - Page 2/2 - Similarity_cells 107415/107415
2023-08-15 20:03:18,145 - INFO - 107415 Total Similarity_cells 


In [46]:
ctx.user_repository.find()

2023-08-15 20:05:51,021 - INFO - HTTP Request: GET http://recsys.ddns.net:8000/api/users/?offset=0&limit=100000&username=&email= "HTTP/1.1 200 OK"
2023-08-15 20:05:51,022 - INFO - Page 2/1 - Users 4/4
2023-08-15 20:05:51,022 - INFO - 4 Total Users 


[{
     "id": 1,
     "username": "adrian",
     "email": "adrianmarino1@gmail.com"
 },
 {
     "id": 5000,
     "username": "adrian.marino",
     "email": "adrian.marino@pedidosya.com"
 },
 {
     "id": 5001,
     "username": "Lionel5000",
     "email": "leonardogrosso@gmail.com"
 },
 {
     "id": 5002,
     "username": "adrianmarino",
     "email": "adrianmarino@gmail.com"
 }]

In [61]:
df = pd.DataFrame(rows)
distances = df[['row', 'column', 'value']].rename(columns={'row': 'user_a', 'column': 'user_b', 'value':'distance'})
distances

,user_a,user_b,distance
0,4,290,0.959354
1,4,156,0.955594
2,4,202,0.954636
3,4,57,0.954088
4,4,221,0.954016
...,...,...,...
107410,584,1,0.431800
107411,14,1,0.431654
107412,94,1,0.430807
107413,353,1,0.427067


In [62]:
distances[distances.user_a == 5000]

,user_a,user_b,distance
106952,5000,1,0.956274
107029,5000,1,0.956274


In [79]:
top_50_most_similar_user_ids = distances[(distances.user_b == 5000) & (distances.distance > 0)] \
    .sort_values(by=['distance']) \
    .head(50)['user_a'] \
    .values

top_50_most_similar_user_ids

array([136,  94, 353,  14, 121, 102, 584, 126, 142, 485, 604, 379, 321,
       498, 134, 447, 468, 130,  99, 242, 536,  46, 181, 107, 588,   8,
       386,  43,   6, 512, 411, 530,  56, 609, 109, 458, 174, 179, 374,
        58, 373, 446, 436, 602, 395, 541, 444, 470, 170, 347])